### <center> Download XAU Data e Armazenamento</center>

--------

#### Bibliotecas

In [1]:
from pathlib import Path
from minio import Minio
from shutil import copy2
import kagglehub

/home/gh057/.cache/pypoetry/virtualenvs/xau-usd-streamlit-0XIE9W8c-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Variáveis Locais

In [2]:
# Variáveis MinIO
MINIO_ENDPOINT = "localhost:9000"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
BUCKET_NAME = "xau-lake"

In [3]:
# Variável KaggleHub
DATASET_KAGGLE = "novandraanugrah/xauusd-gold-price-historical-data-2004-2024"

In [4]:
# Diretórios base do projeto
BASE_DIR = Path("..") / "data"

In [5]:
# Camada Bronze (local)
BRONZE_RAW_DIR = BASE_DIR / "raw"
BRONZE_RAW_DIR.mkdir(parents=True, exist_ok=True)

#### Conectando ao MinIO e Criando/Conferindo Bucket

In [6]:
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False,
)

In [7]:
if not client.bucket_exists(BUCKET_NAME):
    client.make_bucket(BUCKET_NAME)
    print(f"Bucket '{BUCKET_NAME}' criado.")
else:
    print(f"Bucket '{BUCKET_NAME}' já existe.")

Bucket 'xau-lake' criado.


#### Baixando os Dados

In [8]:
dataset_path = kagglehub.dataset_download(DATASET_KAGGLE)

#print("Path to dataset files:", dataset_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96.9M/96.9M [00:16<00:00, 6.20MB/s]

Extracting files...


In [9]:
p = Path(dataset_path)
#list(p.iterdir())

#### Copiando dados para Pasta do Projeto

In [10]:
for csv_path in p.glob("XAU_*_data.csv"):
    destino = BRONZE_RAW_DIR / csv_path.name
    copy2(csv_path, destino)
    print(f"Copiado: {csv_path.name} -> {destino}")

Copiado: XAU_1m_data.csv -> ../data/raw/XAU_1m_data.csv
Copiado: XAU_1Month_data.csv -> ../data/raw/XAU_1Month_data.csv
Copiado: XAU_5m_data.csv -> ../data/raw/XAU_5m_data.csv
Copiado: XAU_30m_data.csv -> ../data/raw/XAU_30m_data.csv
Copiado: XAU_4h_data.csv -> ../data/raw/XAU_4h_data.csv
Copiado: XAU_1h_data.csv -> ../data/raw/XAU_1h_data.csv
Copiado: XAU_15m_data.csv -> ../data/raw/XAU_15m_data.csv
Copiado: XAU_1w_data.csv -> ../data/raw/XAU_1w_data.csv
Copiado: XAU_1d_data.csv -> ../data/raw/XAU_1d_data.csv


#### Subindo os Dados no MinIO (Bronze)

In [11]:
for csv_path in BRONZE_RAW_DIR.glob("XAU_*_data.csv"):
    # Ex.: XAU_15m_data.csv -> ["XAU", "15m", "data"] -> "15m"
    parts = csv_path.stem.split("_")
    timeframe = parts[1]

    object_name = f"bronze/timeframe={timeframe}/{csv_path.name}"

    print(f"Enviando {csv_path.name} como {object_name}...")

    client.fput_object(
        BUCKET_NAME,
        object_name,
        str(csv_path)
    )

print("Upload Bronze concluído.")

Enviando XAU_1m_data.csv como bronze/timeframe=1m/XAU_1m_data.csv...
Enviando XAU_1Month_data.csv como bronze/timeframe=1Month/XAU_1Month_data.csv...
Enviando XAU_5m_data.csv como bronze/timeframe=5m/XAU_5m_data.csv...
Enviando XAU_30m_data.csv como bronze/timeframe=30m/XAU_30m_data.csv...
Enviando XAU_4h_data.csv como bronze/timeframe=4h/XAU_4h_data.csv...
Enviando XAU_1h_data.csv como bronze/timeframe=1h/XAU_1h_data.csv...
Enviando XAU_15m_data.csv como bronze/timeframe=15m/XAU_15m_data.csv...
Enviando XAU_1w_data.csv como bronze/timeframe=1w/XAU_1w_data.csv...
Enviando XAU_1d_data.csv como bronze/timeframe=1d/XAU_1d_data.csv...
Upload Bronze concluído.
